# 8-9. 프로젝트 - Movielens 영화 추천 실습

이번 프로젝트는 MNIST의 Movielens 데이터를 활용하여 영화 추천 서비스를 구동해보는 프로젝트이다.   
아래와 같이 데이터를 다운로드하고, 디렉토리를 준비한다.

`$ wget http://files.grouplens.org/datasets/movielens/ml-1m.zip`   
`$ mv ml-1m.zip ~/aiffel/recommendata_iu/data`   
`$ cd ~/aiffel/recommendata_iu/data`   
`$ unzip ml-1m.zip`

## 1) 데이터 준비와 전처리

In [3]:
import pandas as pd
import os

In [4]:
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python')
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [5]:
ratings['movie_id'].nunique()

3706

**ratings** 파일을 확인해보네, ID, 영화_ID, rating, timestamp의 데이터를 가지고 있다. **timestamp**에 대해서는 구글링을 해봤는데 어떤걸 의미하는지 찾지 못했다..

In [6]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')


orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


**rating**이 3점 이상인 데이터들만 `ratings`라는 변수에 담아두고, formatting을 활용해서 데이터를 확인해본다.

In [7]:
# rating 컬럼의 이름을 count로 바꿉니다.
ratings.rename(columns={'rating':'count'}, inplace=True)

`pandas`의 `rename`을 사용하여 **rating** column 이름을 **count**로 바꿔준다. 

In [8]:
ratings['count']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: count, Length: 836478, dtype: int64

값을 확인해 봤을 떄, 1000208번째까지 값이 나와 확인했다. 분명히 위에서 3점 이상은 filter을 해줬었다.   
밑에 값을 보니 Length는 제대로 나온것 같고, 인덱스 번호인것 같다.

In [9]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding = 'ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


**movies.dat**에는 영화제목과, genre값이 들어있었다.   
궁금한점은, 평점과 영화 제목을 어떻게 맞출지...?가 고민이다

In [10]:
movies['movie_id'].nunique()

3883

길이가 다른데,,,, 어떻게 맞춰질까..?

위에 대한 해답을 찾기 위해 여러가지 시도를 해보겠다.   
먼저, **rating**에 있는 **movie_id**를 sort 해준다.

In [11]:
sorted_ratings = ratings.sort_values(by=['movie_id', 'count'])

In [12]:
sorted_ratings

,user_id,movie_id,count,timestamp
2530,21,1,3,978139347
3405,26,1,3,978130703
3847,28,1,3,978985309
9852,68,1,3,991376026
10126,73,1,3,977867812
...,...,...,...,...
745205,4448,3952,5,984802874
784316,4682,3952,5,998535231
801606,4802,3952,5,988285919
887055,5359,3952,5,971501252


**movie** 파일도 동일하게 **movie_id**을 기준으로 sort해준다.

In [13]:
sorted_title = movies.sort_values(by='movie_id')

In [14]:
sorted_title

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
3878,3948,Meet the Parents (2000),Comedy
3879,3949,Requiem for a Dream (2000),Drama
3880,3950,Tigerland (2000),Drama
3881,3951,Two Family House (2000),Drama


In [15]:
movie = pd.merge(sorted_ratings, sorted_title, on='movie_id')

In [16]:
movie.head(20)

,user_id,movie_id,count,timestamp,title,genre
0,21,1,3,978139347,Toy Story (1995),Animation|Children's|Comedy
1,26,1,3,978130703,Toy Story (1995),Animation|Children's|Comedy
2,28,1,3,978985309,Toy Story (1995),Animation|Children's|Comedy
3,68,1,3,991376026,Toy Story (1995),Animation|Children's|Comedy
4,73,1,3,977867812,Toy Story (1995),Animation|Children's|Comedy
5,80,1,3,977786904,Toy Story (1995),Animation|Children's|Comedy
6,90,1,3,993872933,Toy Story (1995),Animation|Children's|Comedy
7,99,1,3,982873678,Toy Story (1995),Animation|Children's|Comedy
8,114,1,3,977506130,Toy Story (1995),Animation|Children's|Comedy
9,117,1,3,977498304,Toy Story (1995),Animation|Children's|Comedy


`pandas`의 `merge` 메서드를 이용하여 **movie_id**를 기준으로 dataframe형태로 묶어주었다.

## 3) 내가 선호하는 영화를 5가지 골라서 rating에 추가해 줍시다.

In [25]:
my_favorite = ['Batman', 'Joker' ,'Iron man' ,'Avengers' ,'X-men']

# 'zimin'이라는 user_id가 위 아티스트의 노래를 30회씩 들었다고 가정하겠습니다.

my_playlist = pd.DataFrame({'user_id': ['ebang']*5, 'title': my_favorite, 'genre' : ['Action']*5, 'count':[5]*5})
                                                       
                            
if not movie.isin({'user_id':['ebang']})['user_id'].any():  # user_id에 'zimin'이라는 데이터가 없다면
    movie = movie.append(my_playlist)                           # 위에 임의로 만든 my_favorite 데이터를 추가해 줍니다. 

In [26]:
movie.head(-5)

,user_id,movie_id,count,timestamp,title,genre
0,21,1.0,3,978139347.0,Toy Story (1995),Animation|Children's|Comedy
1,26,1.0,3,978130703.0,Toy Story (1995),Animation|Children's|Comedy
2,28,1.0,3,978985309.0,Toy Story (1995),Animation|Children's|Comedy
3,68,1.0,3,991376026.0,Toy Story (1995),Animation|Children's|Comedy
4,73,1.0,3,977867812.0,Toy Story (1995),Animation|Children's|Comedy
...,...,...,...,...,...,...
0,ebang,NaN,5,NaN,Batman,Action
1,ebang,NaN,5,NaN,Joker,Action
2,ebang,NaN,5,NaN,Iron man,Action
3,ebang,NaN,5,NaN,Avengers,Action


내가 좋아하는 영화 5가지와, 장르, rating을 추가하였다   
가장 나중에 추가하였으니 가장 뒷부분에 내가 좋아하는 영화가 추가되었다.

## 4) CSR matrix를 직접 만들어 봅시다.

In [27]:
user_unique = movie['user_id'].unique()
title_unique = movie['title'].unique()

print(len(user_unique))
print(len(title_unique))

6040
3633


CSR matrix를 만들기 위해 먼저, `user_id`와 `title`의 unique값을 구했더니 각각 6040개, 3633개가 나온다.

In [28]:
user_to_idx = {v:k for k,v in enumerate(user_unique)}
title_to_idx = {v:k for k,v in enumerate(title_unique)}

키값과 벨류값을 바꿔 user이름이나 title이름으로 쉽게 찾을 수 있도록 한다.

In [29]:
print(user_to_idx['ebang'])
print(title_to_idx['Batman'])

6039
3628


In [30]:
temp_user_data = movie['user_id'].map(user_to_idx.get)
if len(temp_user_data) == len(movie):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    movie['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# artist_to_idx을 통해 artist 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_title_data = movie['title'].map(title_to_idx.get)
if len(temp_title_data) == len(movie):
    print('title column indexing OK!!')
    movie['title'] = temp_title_data
else:
    print('title column indexing Fail!!')

movie

user_id column indexing OK!!
title column indexing OK!!


,user_id,movie_id,count,timestamp,title,genre
0,0,1.0,3,978139347.0,0,Animation|Children's|Comedy
1,1,1.0,3,978130703.0,0,Animation|Children's|Comedy
2,2,1.0,3,978985309.0,0,Animation|Children's|Comedy
3,3,1.0,3,991376026.0,0,Animation|Children's|Comedy
4,4,1.0,3,977867812.0,0,Animation|Children's|Comedy
...,...,...,...,...,...,...
0,6039,NaN,5,NaN,3628,Action
1,6039,NaN,5,NaN,3629,Action
2,6039,NaN,5,NaN,3630,Action
3,6039,NaN,5,NaN,3631,Action


이 부분에서 `dropna()`를 사용해서 길이가 같을 경우에만 **title**을 인덱싱 하도록하니, 정상적으로 인덱싱되지 않은 값들이 있어서 그런지 오류가 났다. 그 값을 찾기가 너무 복잡해서 그냥 dropna를 제거하고 진행하였다.

In [31]:
from scipy.sparse import csr_matrix

num_user = movie['user_id'].nunique()
num_title = movie['title'].nunique()

csr_data = csr_matrix((movie['count'], (movie['user_id'], movie['title'])), shape= (num_user, num_title))
csr_data

<6040x3633 sparse matrix of type '<class 'numpy.longlong'>'
	with 836483 stored elements in Compressed Sparse Row format>

내 경우에는 **csr matrix**가 6040 x 3363의 형태로 만들어졌다.

In [32]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [33]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

`AlternatingLeastSquares` 모델을 사용하여 추천 시스템을 구동시킨다.   
파라미터로는 factor(차원값)을 100으로 줬고, 정규화값은 0.01, gpu는 사용하지 않고, iterations(epochs)은 15번으로 설정하였다.

In [34]:
csr_data_transpose = csr_data.T
csr_data_transpose

<3633x6040 sparse matrix of type '<class 'numpy.longlong'>'
	with 836483 stored elements in Compressed Sparse Column format>

als 모델은 인풋값으로 (item X user) 형태의 matrix를 받기 때문에 Transpose를 해주었다.

In [66]:
als_model.fit(csr_data_transpose)

위와 같이 모델을 설정하고, 학습을 시켰다.

In [67]:
ebang, Batman = user_to_idx['ebang'], title_to_idx['Batman']
ebang_vector, Batman_vector = als_model.user_factors[ebang], als_model.item_factors[Batman]

In [68]:
ebang_vector

array([ 3.33411423e-12,  1.35713567e-11,  7.79420799e-12,  1.04606341e-11,
       -6.50933387e-12, -5.47929063e-12,  5.08541603e-12, -2.90178580e-13,
       -1.92322980e-12,  2.10961305e-12, -3.12875680e-12, -3.18426839e-12,
       -5.11162889e-13, -1.06536412e-11, -3.10259804e-12,  1.58013186e-11,
        1.73634874e-11,  5.13603483e-12, -2.60826690e-12, -1.95259330e-11,
        5.91238809e-13, -3.55208246e-12,  8.31320689e-13, -1.87667035e-11,
        3.54679784e-12,  1.68501888e-12,  5.22432531e-12, -4.85888510e-13,
       -1.98733608e-12, -1.66059926e-11, -8.20772877e-12,  5.89972775e-12,
        2.52486617e-12, -5.65965027e-12,  5.58885273e-12,  6.22021965e-12,
       -2.56825876e-12,  2.32231899e-12,  2.34159827e-12,  9.03030472e-13,
        1.04477408e-11,  7.55562019e-12, -1.74406461e-11, -3.59646297e-12,
       -7.30242602e-12,  1.94645428e-12, -1.26569024e-11,  1.30882041e-11,
       -1.74543557e-11, -4.31283525e-12,  1.06445321e-11,  5.96497114e-12,
        8.00865190e-12, -

In [69]:
Batman_vector

array([ 4.81531793e-14,  1.21311815e-13,  6.22590426e-14,  1.02673600e-13,
       -3.95481038e-14, -1.21482067e-14,  6.49330917e-14,  1.41667277e-14,
        2.22655299e-14,  4.23853085e-14, -3.14611821e-14,  4.78830673e-14,
        5.13141369e-14, -7.81637599e-14,  5.58647502e-14,  1.30144972e-13,
        1.47254889e-13,  6.15607961e-14, -1.21118876e-14, -1.04118618e-13,
        3.27203407e-14,  4.82599182e-15,  4.39492599e-14, -4.79990600e-14,
        5.27648807e-14,  4.59565213e-14,  5.39805932e-14,  1.71910995e-14,
       -1.24350501e-14, -3.86211110e-14, -7.69266581e-14,  4.54100021e-14,
        3.61203851e-14,  2.58366343e-14,  7.64100561e-14,  8.51533538e-14,
        2.43091404e-15,  4.20788892e-14,  4.43894562e-14,  3.76532810e-14,
        8.29934401e-14,  5.95628622e-14, -9.76370609e-14,  2.27349911e-14,
       -3.75511932e-14,  4.35711478e-14, -6.84672588e-14,  1.24109409e-13,
       -8.88233375e-14, -1.08950087e-14,  9.74930518e-14,  7.95890250e-14,
        9.22228128e-14,  

In [70]:
np.dot(ebang_vector, Batman_vector)

3.951218e-23

먼저, user_id에서 내 아이디와, 내가 고른 영화 **Batman**을 확인해보니 3.9512... 의 값이 나온다.   
노드에서는 1을 기준으로 했던것과 달리 나는 더 큰 값이 나왔다. 정규화가 제대로 되지 않은걸까.. 어떤 오류인지 확인하지 못하고 그대로 진행해보았다.

In [72]:
joker = title_to_idx['Joker']
joker_vector = als_model.item_factors[joker]
np.dot(ebang_vector, joker_vector)

6.1335955e-23

조커와의 거리는 6이 넘게 나왔다.

In [73]:
favorite_movie = 'Batman'
movie_id = title_to_idx[favorite_movie]
similar_movie = als_model.similar_items(movie_id, N=15)
similar_movie

[(3628, 0.99999994),
 (3629, 0.98597646),
 (3632, 0.9842743),
 (3631, 0.96867067),
 (3630, 0.9678733),
 (2947, 0.7392987),
 (131, 0.73929864),
 (564, 0.73929864),
 (2566, 0.7392985),
 (657, 0.7392985),
 (1992, 0.7392931),
 (1395, 0.73929304),
 (1363, 0.7392528),
 (586, 0.73925275),
 (710, 0.73925275)]

In [74]:
#artist_to_idx 를 뒤집어, index로부터 artist 이름을 얻는 dict를 생성합니다. 
idx_to_title = {v:k for k,v in title_to_idx.items()}
[idx_to_title[i[0]] for i in similar_movie]

['Batman',
 'Joker',
 'X-men',
 'Avengers',
 'Iron man',
 "Another Man's Poison (1952)",
 'Neon Bible, The (1995)',
 "I Don't Want to Talk About It (De eso no se habla) (1993)",
 'Male and Female (1919)',
 'Daens (1992)',
 'Number Seventeen (1932)',
 'War at Home, The (1996)',
 "Brother's Kiss, A (1997)",
 'Century (1993)',
 'Last of the High Kings, The (a.k.a. Summer Fling) (1996)']

최종적으로 배트맨과 비슷한 부류의 영화를 추천받아보니 내가 좋아서 추가하였던 Joker, X-men, Avengers, Iron man등이 가장 먼저 나오고, 이후에는 드라마 종류의 영화가 추천되었다.

In [78]:
def get_similar_movie(movie_name: str):
    movie_id = title_to_idx[favorite_movie]
    similar_movie = als_model.similar_items(movie_id, N=15)
    similar_movie = [idx_to_title[i[0]] for i in similar_movie]
    return similar_movie

위 과정을 전부 함수로 만들어주고,

In [79]:
get_similar_movie('X-men')

['Batman',
 'Joker',
 'X-men',
 'Avengers',
 'Iron man',
 "Another Man's Poison (1952)",
 'Neon Bible, The (1995)',
 "I Don't Want to Talk About It (De eso no se habla) (1993)",
 'Male and Female (1919)',
 'Daens (1992)',
 'Number Seventeen (1932)',
 'War at Home, The (1996)',
 "Brother's Kiss, A (1997)",
 'Century (1993)',
 'Last of the High Kings, The (a.k.a. Summer Fling) (1996)']

X-men과 비슷한 부류의 영화를 추천해달라고하니, Batman, Joker, Avengers 등의 영화를 추천해주는걸 확인할 수 있다.

# 회고:  
## 1. **이번 프로젝트에서 어려웠던 점**   
세세한 부분을 확실히 이해하지 못해서 프로젝트를 진행하는데 어려움이 있었다. 전반적인 흐름을 알겠으나, 각 과정에서 디테일하게 이해를 하지 못하면 오류가 발생했을 시 바로잡기가 더욱 힘들어진다. 이번 프로젝트에서는 4)CSR matrix를 만드는 과정에서 오류가 조금 있었는데 어느정도 대처를 하고 넘어가 다행히 프로젝트를 마무리 할수 있게 되었다. 이처럼 전체 과정에서 세세한 부분을 이해하지 못해서 생기는 문제가 개인적으로는 어려웠다.

   
## 2. **프로젝트를 진행하면서 알아낸 점 혹은 아직 모호한 점**   
결국에는 한 벡터값을 기준으로 거리가 가까운 다른 벡터를 비교해 추천해주는 방식으로 이 추천서비스가 작동되는 것을 이해했다. 하지만 그외에 왜 AlternatingLeastSquares 모델이 사용되었는지에 대해 설명이 안되어 있어 아쉽다.

## 3. **루브릭 평가 지표를 맞추기 위해 시도한 것들**   
- CSR matrix가 정상적으로 만들어졌다.   
3633x6040 sparse matrix를 생성하였다.   
- MF 모델이 정상적으로 훈련되어 그럴듯한 추천이 이루어졌다.   
AlternatingLeastSquares을 사용하여 모델을 세팅하였고, 트레인 하였다.   
- 비슷한 영화 찾기와 유저에게 추천하기의 과정이 정상적으로 진행되었다.   
Batman과 비슷한 액션 영화를 찾을 수 있었으며 비슷한 영화 15종을 추천해주었다.

   
### 4. **만약에 루브릭 평가 관련 지표를 달성 하지 못했을 때, 이유에 관한 추정**   
위 1번에서 말한 부분과 연관된 부분인데, 전체 과정을 이해하지 못했기 보다는 세부적인 내용에 대해 이해가 안가는 부분이 있어서 오류가 생겼을 시 해결하는데 어려움이 있었을것 같다. 지금도 당장은 해결하고 프로젝트를 제출하는데는 문제가 없지만, 근본적인 해결책을 통하여 해결하지 못한거 같아 아쉬움이 있다.
   
### 5. **자기 다짐**   
추천시스템은 사실 안쓰이는 플랫폼이 없을 정도로 널리 사용되고 있는데 제대로 이해하지 못하고 프로젝트를 제출하게되서 아쉬운 마음이다. 오프라인 이었다면 팀원들과 모여 질문하고 토의하며 궁금증을 해결했을 수도 있었을텐데라는 아쉬움도 있다. 잘 만들어진 추천서비스는 상업적으로 꽤나 유용한 프로그램이기에 나중에 기회가 된다면 더 공부해볼 예정이다.


   